In [ ]:
%pip install --upgrade --quiet langchain_community langchain-openai langchainhub 
%pip install --upgrade --quiet  azure-search-documents>=11.4
%pip install --upgrade --quiet  azure-identity
%pip install python-dotenv

In [1]:
import os
import dotenv

dotenv.load_dotenv()

ai_serch_svc = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
ai_search_index = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")
phi_key = os.getenv("PHI_SERVERLESS_KEY")
phi_url = os.getenv("PHI_SERVERLESS_URL")
llama_key = os.getenv("LLAMA_MANAGED_KEY")
llaam_url = os.getenv("LLAMA_MANAGED_URL")

Sample Question:

1. I’m an ICU nurse looking to move to Nanaimo, do you have any openings?
* Follow up question: what’s it like living in Nanaimo?
* Follow up question: what’re house prices like?
* what would I need to qualify for one of those roles?

#### Create Azure AI Search Retriever

In [2]:
from langchain_community.retrievers import AzureAISearchRetriever

retriever = AzureAISearchRetriever(
    api_key=ai_search_api_key,
    index_name=ai_search_index,
    service_name=ai_serch_svc,
    top_k=5,
    content_key="chunk"
)

#### Prompt

In [3]:

from langchain import hub
prompt = hub.pull("langchain-ai/retrieval-qa-chat")

q1 = "I am an ICU nurse looking to move to Nanaimo, do you have any openings?"
q2 = "Do I qualify for this role if I have 5 years of experience as a critical care nurse?"
q3 = "How is it like living in that city?"
q4 = "What is the salary range for this role?"

chat_history = []

#### GPT 4-o

In [4]:
from langchain_openai import AzureChatOpenAI

gpt4o = AzureChatOpenAI(
    api_version="2024-06-01",
    azure_deployment="gpt-4o",
    temperature=0,
    top_p=0.95,
    max_tokens=1000,
)

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage

gpt4o_retrieval_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(gpt4o, prompt))

In [6]:
gptr1 = gpt4o_retrieval_chain.invoke({"input": q1, "chat_history": chat_history})
print("\n" + gptr1["answer"])
chat_history.extend([HumanMessage(content=q1), gptr1["answer"]])


Yes, there are current and future opportunities for Registered Nurses at Nanaimo Regional General Hospital (NRGH) in various departments, including the Intensive Care Unit (ICU). You can express your interest by applying to the job postings listed under the Expression of Interest (EOI) numbers JD4916 and JD5029. These positions are part of Island Health, which is the Health Authority for Vancouver Island. 

For the ICU & High Acuity Unit specifically, you would need to have:
- Registration with the BC College of Nurses and Midwives as a practicing RN registrant.
- Minimum 2 years of recent Critical Care nursing experience or an accredited Critical Care course.
- Demonstrated ability in arrhythmia interpretation as evidenced by a mark of 75% on the VIHA ECG Challenge exam.
- Advanced Cardiac Life Support and CRRT (continuous renal replacement therapy) are considered assets.

Relocation assistance may be available to qualified applicants. If you meet these qualifications and are interes

In [7]:
gptr2 = gpt4o_retrieval_chain.invoke({"input": q2, "chat_history": chat_history})
print("\n" + gptr2["answer"])
chat_history.extend([HumanMessage(content=q2), gptr2["answer"]])


Based on the job description for the ICU/HAU Critical Care Nurse at Nanaimo Regional General Hospital, you would qualify for the role if you have 5 years of experience as a critical care nurse. The qualifications for this position include:

- Registration with the BC College of Nurses and Midwives as a practicing RN registrant.
- Minimum 2 years of recent Critical Care nursing experience or an accredited Critical Care course.
- Demonstrated ability in arrhythmia interpretation as evidenced by a mark of 75% on the VIHA ECG Challenge exam.
- Advanced Cardiac Life Support and CRRT (continuous renal replacement therapy) are considered assets.

Since you have 5 years of experience in critical care, you meet the experience requirement. If you also meet the other qualifications, such as registration with the BC College of Nurses and Midwives and the ability to interpret arrhythmias, you would be a strong candidate for the position.


In [8]:
gptr3 = gpt4o_retrieval_chain.invoke({"input": q3, "chat_history": chat_history})
print("\n" + gptr3["answer"])
chat_history.extend([HumanMessage(content=q3), gptr3["answer"]])


Nanaimo, located on central Vancouver Island, offers a unique blend of natural beauty, recreational opportunities, and a vibrant community. Here are some aspects of living in Nanaimo:

### Natural Beauty and Outdoor Activities
- **Scenic Views**: Nanaimo is a harbour city with stunning ocean views and easy access to beaches. You can watch cruise ships pass by or enjoy the waterfront.
- **Outdoor Activities**: The city is surrounded by hiking and biking trails, making it a great place for outdoor enthusiasts. There are also opportunities for water sports, fishing, and exploring nearby parks.

### Community and Lifestyle
- **Close-Knit Community**: Nanaimo has a welcoming and close-knit community. It's a place where you can easily get to know your neighbors and feel a sense of belonging.
- **Cultural Opportunities**: The city offers various cultural and recreational activities, including local markets, festivals, and community events.

### Accessibility
- **Convenient Location**: Nanaim

In [9]:
print("\n" + gpt4o_retrieval_chain.invoke({"input": q4, "chat_history": chat_history})["answer"])
chat_history.clear()


The salary range for a Staff Nurse, including those in specialized units like the Intensive Care Unit (ICU), is determined by the Nurses Bargaining Association and follows the pay scale outlined in the collective agreement. For a position classified under Direct Patient Care - Profile Classification - Level 3 (NL3), the salary grid typically ranges from approximately $35 to $47 per hour, depending on factors such as years of experience and specific qualifications.

For more precise information, you would need to refer to the current collective agreement or contact the Human Resources department of Island Health. They can provide the exact salary range based on your experience and qualifications.


#### Phi - Managed endpoint

- Phi 3 medium 128K
The allowed skus for this model are 
1. Standard_NC48ads_A100_v4 - $6616.59 per month / 3 year reserved
2. Standard_NC96ads_A100_v4
3. Standard_ND96amsr_A100_v4

- Phi 3 small 128K
The allowed skus for this model are 
1. Standard_NC24ads_A100_v4 - $3308.29 per month / 3 year reserved
2. Standard_NC48ads_A100_v4
3. Standard_NC96ads_A100_v4
4. Standard_ND96amsr_A100_v4

In [10]:
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    CustomOpenAIChatContentFormatter,
)

phi3s128k_model = AzureMLChatOnlineEndpoint(
    endpoint_url=phi_url,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=phi_key,
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0, "top_p": 0.95, "max_new_tokens": 1000}
)

In [11]:
phi3s128k_retrieval_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(phi3s128k_model, prompt))

In [12]:
phir1 = phi3s128k_retrieval_chain.invoke({"input": q1, "chat_history": chat_history})
print("\n" + phir1["answer"])
chat_history.extend([HumanMessage(content=q1), phir1["answer"]])


Yes, we currently have an opening for a Registered Nurse in the ICU & High Acuity Unit at Nanaimo Regional General Hospital. This position is a Level 3 GRID/PAY LEVEL role, as per the Nurses Bargaining Association HSCIS NO.: 21001, and is represented by the BCNU union. The job involves providing direct patient care in the Intensive Care Unit (ICU) and High Acuity Unit (HAU), with a focus on acutely ill patients. The staffing ratio is 1:1 for ventilated and critically unstable patients. The ICU nurses are part of the Code Blue team, except for the Cowichan District and West Coast General Hospitals. Continuous Renal Replacement Therapy (CRRT) and organ donor maintenance are available at the Victoria and Nanaimo Hospitals. The High Acuity units provide care for adult patients requiring increased monitoring and resuscitation in a multidisciplinary environment. The nurse to patient ratio in this unit is 1:2. The position requires a minimum of 2 years recent Critical Care nursing experience

In [13]:
phir2 = phi3s128k_retrieval_chain.invoke({"input": q2, "chat_history": chat_history})
print("\n" + phir2["answer"])
chat_history.extend([HumanMessage(content=q2), phir2["answer"]])


Yes, with 5 years of experience as a critical care nurse, you would be well-qualified for the Registered Nurse position in the ICU & High Acuity Unit at Nanaimo Regional General Hospital. The job description specifies a minimum of 2 years recent Critical Care nursing experience, and your 5 years of experience would exceed this requirement. Additionally, the job description mentions that Advanced Cardiac Life Support and CRRT (continuous renal replacement therapy) are assets, which suggests that these skills would be beneficial but not necessarily required. However, it's always a good idea to check the specific job posting for any additional qualifications or requirements.


In [14]:
phir3 = phi3s128k_retrieval_chain.invoke({"input": q3, "chat_history": chat_history})
print("\n" + phir3["answer"])
chat_history.extend([HumanMessage(content=q3), phir3["answer"]])


Nanaimo, located on central Vancouver Island, is a vibrant city with a rich cultural heritage and a variety of recreational opportunities. It's a convenient access point to the mainland and is known for its beautiful harbour and beach. The city is also a popular stop for cruise ships, adding to its lively atmosphere. In terms of healthcare, Nanaimo Regional General Hospital is a major employer and provides a range of services, including a Level 3 ICU & High Acuity Unit. The city is also home to a number of educational institutions, making it a great place for families. If you're interested in the job, you might also want to consider the city's cost of living, climate, and other lifestyle factors.


In [15]:
print("\n" + phi3s128k_retrieval_chain.invoke({"input": q4, "chat_history": chat_history})["answer"])
chat_history.clear()


The salary for the Registered Nurse position in the ICU & High Acuity Unit at Nanaimo Regional General Hospital is determined by the Level 3 GRID/PAY LEVEL as per the Nurses Bargaining Association HSCIS NO.: 21001. The exact salary range can vary based on factors such as experience, education, and specific skills. However, as a Level 3 position, you can expect a competitive salary that reflects the high level of responsibility and expertise required for this role. For the most accurate and up-to-date salary information, I would recommend checking the specific job posting or contacting the hospital's human resources department.


#### LLama - Managed endpoint

- LLama 3.1 70B

The allowed skus for this model are
1. Standard_NC96ads_A100_v4
2. Standard_ND96asr_v4 
3. Standard_ND96amsr_A100_v4

- LLama 3.1 8B

The allowed skus for this model are 
1. Standard_ND40rs_v2
2. Standard_NC24ads_A100_v4
3. Standard_NC48ads_A100_v4
4. Standard_NC96ads_A100_v4
5. Standard_ND96amsr_A100_v4

In [16]:
llama318k_model = AzureMLChatOnlineEndpoint(
    endpoint_url=llaam_url,
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key=llama_key,
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0, "top_p": 0.95, "max_new_tokens": 1000}
)

In [17]:
llama_retrieval_chain = create_retrieval_chain(retriever, create_stuff_documents_chain(llama318k_model, prompt))

In [18]:
llamar1 = llama_retrieval_chain.invoke({"input": q1, "chat_history": chat_history})
print("\n" + llamar1["answer"])
chat_history.extend([HumanMessage(content=q1), llamar1["answer"]])


According to the job descriptions provided, there are openings for ICU/HAU Critical Care Nurses in Nanaimo, specifically at Nanaimo Regional General Hospital. The job description (JD3027) mentions that the position is available at Nanaimo Regional General Hospital, which is one of the facilities listed.

However, I would recommend checking the Island Health website or contacting the Human Resources department directly to confirm the availability of positions and to inquire about the application process. They may also have more information about the specific requirements and qualifications for the position.

Additionally, you can also check the Island Health website for other ICU nurse positions that may be available in Nanaimo or other locations on Vancouver Island.


In [19]:
llamar2 = llama_retrieval_chain.invoke({"input": q2, "chat_history": chat_history})
print("\n" + llamar2["answer"])
chat_history.extend([HumanMessage(content=q2), llamar2["answer"]])


According to the job description (JD3027), the minimum requirement for the ICU/HAU Critical Care Nurse position is 2 years of recent, related experience, or an equivalent combination of education, training, and experience.

Since you have 5 years of experience as a critical care nurse, you likely meet the experience requirement. However, it's always best to review the job description and requirements carefully to ensure you meet all the qualifications.

Additionally, the job description mentions that Advanced Cardiac Life Support (ACLS) and CRRT (continuous renal replacement therapy) are assets, but not necessarily required. It's possible that having 5 years of experience as a critical care nurse may have provided you with the necessary skills and knowledge to perform these tasks.

To confirm your eligibility, I would recommend reviewing the job description and requirements again, and then contacting the Human Resources department directly to inquire about the application process and 

In [20]:
llamar3 = llama_retrieval_chain.invoke({"input": q3, "chat_history": chat_history})
print("\n" + llamar3["answer"])
chat_history.extend([HumanMessage(content=q3), llamar3["answer"]])


Nanaimo is a city located on the east coast of Vancouver Island, British Columbia, Canada. It's a charming city with a population of around 90,000 people. Here are some aspects of living in Nanaimo:

Pros:

1. **Natural beauty**: Nanaimo is surrounded by mountains, forests, and waterways, making it a great place for outdoor enthusiasts. The city has many parks, trails, and beaches to explore.
2. **Mild climate**: Nanaimo has a mild oceanic climate, with mild winters and cool summers. The average temperature ranges from 2°C to 22°C (36°F to 72°F) throughout the year.
3. **Affordable cost of living**: Compared to other cities in British Columbia, Nanaimo has a relatively low cost of living. Housing, food, and transportation costs are lower than in Vancouver or Victoria.
4. **Community-oriented**: Nanaimo has a strong sense of community, with many local events and festivals throughout the year. The city has a small-town feel, making it easy to get to know your neighbors and feel connecte

In [21]:
print("\n" + llama_retrieval_chain.invoke({"input": q4, "chat_history": chat_history})["answer"])
chat_history.clear()


The job description (JD3027) for the ICU/HAU Critical Care Nurse position does not provide a specific salary range. However, according to the Collective Agreement between the British Columbia Nurses' Union and the Health Employers Association of British Columbia, the salary range for a Registered Nurse (RN) in the Level 3 classification, which is the classification for this position, is between $43.69 and $63.49 per hour, depending on experience and qualifications.

It's also worth noting that the salary range may vary depending on the specific employer, location, and other factors. If you are interested in learning more about the salary range for this role, I would recommend contacting the Human Resources department directly or checking the Island Health website for more information.
